<a href="https://colab.research.google.com/github/Aditi-T27/API-Tester/blob/main/Copy_of_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install accelerate
!pip install bitsandBytes
!pip install transformers
!pip install -U datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import os
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `ModelFineTune` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ModelFineT

In [ ]:
base_model_id="meta-llama/Llama-2-7b-chat-hf"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, #can comment if double quant not needed
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model= AutoModelForCausalLM.from_pretrained(
    base_model_id, quantization_config=bnb_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/Aditi-T27/Dataset_MJAPI

Cloning into 'Dataset_MJAPI'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 140.11 KiB | 6.37 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset(
    "csv",
    data_files={
        "train": [
            "/content/Dataset_MJAPI/synthetic_dataset (1).csv",
        ]
    },
    split="train"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer= LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
tokenized_train_dataset = []

for sample in train_dataset:
    input_text = sample["input"]
    output_text = sample["output"]
    full_text = f"Q: {input_text} A: {output_text}"


    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding=True  # dynamic padding to longest length in batch
    )
    tokenized_train_dataset.append(tokenized)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.decode(tokenized_train_dataset[18]["input_ids"])

'<s> Q: give me a product_code A: PC-12834</s>'

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config= LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["q_proj","k_proj", "v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model= get_peft_model(model,config)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModel",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        learning_rate=1e-4,
        max_steps=100,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10,

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache=False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pineapple00272 (pineapple00272-projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.358200
20,1.593100
30,1.378100
40,1.857800
50,1.322100
60,1.756600
70,1.922100
80,2.028700
90,1.519800
100,1.673400


TrainOutput(global_step=100, training_loss=1.8409908103942871, metrics={'train_runtime': 408.405, 'train_samples_per_second': 0.979, 'train_steps_per_second': 0.245, 'total_flos': 368930323120128.0, 'train_loss': 1.8409908103942871, 'epoch': 0.4})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id="meta-llama/Llama-2-7b-chat-hf"
nf4Config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, #can comment if double quant not needed
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer= LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)
model= AutoModelForCausalLM.from_pretrained(
    base_model_id, quantization_config=nf4Config, device_map="auto", trust_remote_code=True, use_auth_token=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer= LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)
modelFinetuned= PeftModel.from_pretrained(model , "finetunedModel/checkpoint-100")

In [ ]:
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_tokens)


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
['<s>', '</s>', '<unk>']


In [ ]:
# Match your training prompt exactly
user_question = "give me a user_name"
eval_prompt = f"<s> Q: {user_question} A:"


# Tokenize
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")


#Evaluate
modelFinetuned.eval()
# with torch.no_grad():
#     output = modelFinetuned.generate(**promptTokenized, max_new_tokens=5)
#     decoded = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(decoded)
output = model.generate(
    **promptTokenized,
    max_new_tokens=10,  # reasonable limit
    eos_token_id=tokenizer.eos_token_id,  # stop at </s>
    pad_token_id=tokenizer.pad_token_id
)


torch.cuda.empty_cache()

Q: give me a user_name A:02038


In [ ]:
user_question = "give me a message, do not add any numbers or no email or no post_id"
eval_prompt = f"<s> Q: {user_question} A:"

# Tokenize the prompt
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set model to eval mode
modelFinetuned.eval()

# Generate response
output = model.generate(
    **promptTokenized,
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,             # Optional: helps prevent repetitive outputs
    top_p=0.9,
    temperature=0.8
)

# Decode the generated tokens
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print("Model Output:", decoded)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Output: Q: give me a message, do not add any numbers or no email or no post_id A:sanders, and, that, house, a, of, their.

  Q: give me a message A: and, know, find, make, back want see.

  A: game, way people see look


In [ ]:
import torch

# Define your prompts
prompts = [
    "give me a username only, without numbers",
    "give me a comment with only alphabets",
    "give me a post_id",
    "give me an email"
]

# Make sure model is in eval mode
modelFinetuned.eval()

# Loop through each prompt
for user_question in prompts:
    # Match your training prompt format
    eval_prompt = f"<s> Q: {user_question} A:"

    # Tokenize the prompt
    inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        output = modelFinetuned.generate(
            **inputs,
            max_new_tokens=15,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode and print
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Prompt: {user_question}")
    print(f"Output: {decoded}")
    print("-" * 50)

# Clear cache
torch.cuda.empty_cache()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a username only, without numbers
Output: Q: give me a username only, without numbers A:tammyk88
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a comment
Output: Q: give me a comment A:1976658781  A: 2
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a post_id
Output: Q: give me a post_id A:944420630093043
--------------------------------------------------
Prompt: give me an email
Output: Q: give me an email A:tiffany.brown@example.com

Comment: This
--------------------------------------------------


In [ ]:
import torch

# Define your prompts
prompts = [
    "give me a username only, without numbers",
    "give me a status only, without numbers",
    "give me a post_id",
    "give me an email"
]

# Make sure model is in eval mode
modelFinetuned.eval()

# Loop through each prompt
for user_question in prompts:
    # Match your training prompt format
    eval_prompt = f"<s> Q: {user_question} A:"

    # Tokenize the prompt
    inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        output = modelFinetuned.generate(
            **inputs,
            max_new_tokens=15,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode and print
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Prompt: {user_question}")
    print(f"Output: {decoded}")
    print("-" * 50)

# Clear cache
torch.cuda.empty_cache()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a username only, without numbers
Output: Q: give me a username only, without numbers A:tammybrown1174
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a status only, without numbers
Output: Q: give me a status only, without numbers A:: pending
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a post_id
Output: Q: give me a post_id A:024540499997489
--------------------------------------------------
Prompt: give me an email
Output: Q: give me an email A:0004449805@example.com
--------------------------------------------------


In [ ]:
import torch

# Define your prompts
prompts = [
   "Provide a username with lowercase letters only, no numbers, no special characters, no email format.",
    " Provide a status codepos only",
    "Generate a product_code",
    "give me an email"
]

# Make sure model is in eval mode
modelFinetuned.eval()

# Loop through each prompt
for user_question in prompts:
    # Match your training prompt format
    eval_prompt = f"<s> Q: {user_question} A:"

    # Tokenize the prompt
    inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        output = modelFinetuned.generate(
            **inputs,
            max_new_tokens=15,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode and print
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Prompt: {user_question}")
    print(f"Output: {decoded}")
    print("-" * 50)

# Clear cache
torch.cuda.empty_cache()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a user_name only, without numbers or email format
Output: Q: give me a user_name only, without numbers or email format A:_kristen_wood_
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a status only, without numbers
Output: Q: give me a status only, without numbers A:: completed

A: completed

status: completed

comple
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a product_code, without numbers
Output: Q: give me a product_code, without numbers A:000470835599485
--------------------------------------------------
Prompt: give me an email
Output: Q: give me an email A:tiffany.n.kline@example.com

A
--------------------------------------------------


In [ ]:
import torch

# Define your prompts
prompts = [
    "Generate a username with lowercase letters only, no numbers, no special characters, no email format.",
    "give me a status,strictly restrict to one word",
    "Generate a username with lowercase letters only, no numbers, no special characters, no email format.",
    "give me an email"
]

# Make sure model is in eval mode
modelFinetuned.eval()

# Loop through each prompt
for user_question in prompts:
    # Match your training prompt format
    eval_prompt = f"<s> Q: {user_question} A:"

    # Tokenize the prompt
    inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        output = modelFinetuned.generate(
            **inputs,
            max_new_tokens=15,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode and print
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Prompt: {user_question}")
    print(f"Output: {decoded}")
    print("-" * 50)

# Clear cache
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: Generate a username with lowercase letters only, no numbers, no special characters, no email format.
Output: Q: Generate a username with lowercase letters only, no numbers, no special characters, no email format. A:sarahmorgan23
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: give me a status,strictly restrict to one word
Output: Q: give me a status,strictly restrict to one word A:tired

A: tired
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt: Generate a username with lowercase letters only, no numbers, no special characters, no email format.
Output: Q: Generate a username with lowercase letters only, no numbers, no special characters, no email format. A:taylorjennifer50
Bio: I am a student
--------------------------------------------------
Prompt: give me an email
Output: Q: give me an email A:004072843@example.com
--------------------------------------------------


In [ ]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")


In [ ]:
tokenizer.save_pretrained("final_model")

('final_model/tokenizer_config.json',
 'final_model/special_tokens_map.json',
 'final_model/tokenizer.model',
 'final_model/added_tokens.json')

In [ ]:
from peft import PeftModel

modelFinetuned.save_pretrained("final_model")


In [ ]:
modelFinetuned.push_to_hub("Aditi27/FineTunedLLM_Part1")
tokenizer.push_to_hub("Aditi27/FineTunedLLM_Part1")



adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aditi27/FineTunedLLM_Part1/commit/779cce876d45f7f481a3e25d3ea5ae5af827a126', commit_message='Upload tokenizer', commit_description='', oid='779cce876d45f7f481a3e25d3ea5ae5af827a126', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Aditi27/FineTunedLLM_Part1', endpoint='https://huggingface.co', repo_type='model', repo_id='Aditi27/FineTunedLLM_Part1'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import PeftModel

base_model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")  # or your base
tokenizer = LlamaTokenizer.from_pretrained("Aditi27/FineTunedLLM_Part1")

model = PeftModel.from_pretrained(base_model, "Aditi27/FineTunedLLM_Part1")


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import shutil

shutil.make_archive("finetunedModel_checkpoint100", 'zip', "finetunedModel/checkpoint-100")


'/content/finetunedModel_checkpoint100.zip'